## Exercise 2


In [4]:
import math

In [8]:
#Invert a byte
def InvertByte(value):
    newValue = 0
    for i in range(0, 8):
        mask = (1 << i)
        if (mask & value) == 0:
            newValue |= (1 << i);
        
    return newValue
# Return the index of the first zero in the byte, counting left to right
def GetFirstIndexOfZero(byte, startIndex):
    #invert the byte, so as to have 1s where there are 0s. This way I can detect zeros with a & operation
    invByte = InvertByte(byte)
    
    #Create a bit mask with a 1 in each byte position, starting from left to right
    for i in range(0, 8-startIndex):
        mask = 1 << (7 - i - startIndex)
        if (invByte) & mask != 0:
            return i + startIndex
    return -1

# Return a bit mask having numOfBits of zeros on the left. Used to blank out the left part of a byte
def ZeroizeBitsFromLeftMask(numOfBits):
    mask = 0
    # inject 1s in positions I want to keep
    for i in range(0, 8-numOfBits):
        mask |= (1 << i)
    return mask

# return a bit mask having numOfBits of 1s on the right part of the byte
def RightPassMask(numOfBits):
    mask = 0
    for i in range(0, numOfBits):
        mask |= (1 << i)
    return mask

# return the number of significant bits in a byte
def GetBinaryLength(value):
    length = 0
    while(value > 0):
        length += 1
        value >>= 1
    return length


# Encode a single byte using Rice encoding algorithm
# This method returns a string instead of a number.
# I made this choice to manage the case of zero, which is encoded as a sequence of k+1 zeros.
# If returned as integer that sequence would be flatten as 0, losing the information about the length
def RiceEncode(value, k):
    m = math.pow(2, k)
    
    quotient =  int(math.floor(value/m))
    remainder = int(value % m)
    
    # concatenate a set of ones followed by a zero
    quotientCode = ('1' * quotient) + '0'
    remainderCode = format(remainder, f'0{k}b')
    
    encodedValue = quotientCode + remainderCode
    
    return encodedValue
    

# Decode a value using Rice decoding algorithm
def RiceDecode(value, k):

    m = math.pow(2, k)
    
    bQuotient = value >> k + 1

    quotient = GetBinaryLength(bQuotient)
        
    mask = RightPassMask(k)
    remainder = value & mask
    
    decodedValue = int(quotient * m + remainder)
    return decodedValue




# Encode a file using the Rice encoding algorithm
def EncodeFileWithRiceEncode(source, destination, k):
    #Read from source byte by byte.
    #Encode each byte. The encoded value can be longer or shorter than a byte.
    #If it is longer, split it in chunks one-byte long and add it to the buffer
    #If it is shorter, packs it into one byte and fill the remaining bits with the next chunk
    
    # Buffer size for input and output buffers
    bufSize = 262144
    
    #buffer
    outBuffer = bytearray()
    
    #temporary byte where to store bits
    dByte = 0
    
    #bits left empty in the dByte
    bitsLeft = 8


    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:

            # Bulk read from file into input buffer
            inBuffer = sStream.read(bufSize)
            while len(inBuffer):
                # Read from input buffer
                for sByte in inBuffer:

                    #Encode the byte
                    sEncByte = RiceEncode(sByte, k)
                    lenEncByte = len(sEncByte)

                    #index store the current position in the encoded value
                    index = 0
                    #pack the encode value in chunks into dByte
                    while(index < lenEncByte):
                        #read bytes from index to nextIndex, max 8 bits, not going beyong the end of encoded value
                        nextIndex = min(index+bitsLeft, lenEncByte)
                        #how many bits left empty in dByte
                        bitsLeft -= (nextIndex - index)
                        
                        #pack the bits into dByte
                        #If the number of bits to pack fills dByte, there is no bit shift to do
                        #If the number of bits to pack is shorter than the number of bits left, 
                        #shift the bits to the left so as to pack them into the dByte and left the free space on the right side
                        dByte |= (int(sEncByte[index:nextIndex], 2) << bitsLeft)

                        #If there are no bits left, meaning that the dByte is fully packed
                        if(bitsLeft == 0):
                            #add the dByte to the buffer
                            outBuffer.append(dByte)

                            #empty the dByte for next iteration
                            dByte = 0
                            #reset the number of available bits in dByte
                            bitsLeft = 8
                            
                            #Check the buffer, if its full (its size is greater than or equal to the buffers size set in bufSize)
                            if(len(outBuffer) >= bufSize):
                                #write the buffer into the destination stream
                                dStream.write(outBuffer)
                                #empty the buffer
                                outBuffer = bytearray()
                    
                        #Update the index for the next iteration throught the encoded value
                        index = nextIndex
                inBuffer = sStream.read(bufSize)
            # Write the remaining bits in the buffer
            if(bitsLeft != 8):
                outBuffer.append(dByte)
                
            #flush the buffer on disk
            dStream.write(outBuffer)


            
            



# Decode a file using Rice decoding algorithm            
def DecodeFileWithRiceEncode(source, destination, k):

    # Buffer size for input and output buffers
    bufSize = 262144
    
    #buffers
    outBuffer = bytearray()
    byteBuffer = bytearray()
    
    #index = 0
    #counter = 0
    
    # Keeps the current position in the byte extracted from the source
    startIdx = 0
    
    # Keeps the end position of the remainder in the current or next byte 
    shift = 0
    
    #temporary byte where to store bits
    sByte = None
    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            
            # Read first byte from stream
            sByte = sStream.read(1)
            # Read next byte from stream
            nextByte = sStream.read(1)

            # while there is a byte in sByte
            while len(sByte) > 0: 
                
                #counter += 1
                # Find the first zero position in the byte
                indexOfZero = GetFirstIndexOfZero(sByte[0],  startIdx)
                
                # If not found, it is a all-1 byte and can be added to the buffer as is
                if indexOfZero == -1:
                    # add to buffer
                    byteBuffer.append(sByte[0])
                    
                    #read next byte from source
                    sByte = nextByte
                    nextByte = sStream.read(1)

                    #reset the index
                    startIdx = 0;
                    continue
                
                #if zero is found and the end of the remainder part falls off to the next byte
                if indexOfZero + k >= 8:
                    #add the byte to the buffer
                    byteBuffer.append(sByte[0])
                    
                    #read one more byte
                    sByte = nextByte
                    nextByte = sStream.read(1)

                    #end position of the remainder is in the next byte 
                    shift = (8 + 7 - (indexOfZero + k))
                else:
                    #end position of the remainder is in the current byte 
                    shift = 7 - (indexOfZero + k)

                #read the byte as an integer
                sByteAsInt = int.from_bytes(sByte, "big")
                
                #clean leftmost 8-shift bits
                zeroizeMask = ZeroizeBitsFromLeftMask(8-shift)

                #save the bits beyond the end of the remaining part, as belonging to the next value
                #remByte will be added to the next iteration's buffer
                remByte = sByteAsInt & zeroizeMask
                

                #add current byte to bytearray
                #note that the rigthmost portion of the current byte (which belongs to the next value)
                #will be discarded in the next for loop (as already extracted in remByte, being part of the next value)
                byteBuffer.append(sByteAsInt)
                     
                #the next loop repositions the bits in the various bytes constituting the current value
                #basically considering all bytes in the buffer as a unique value and right-shifting all bits by
                #a value given by shift
                #Example (x are bits extracted in remByte):
                # 1st byte: 11111111
                # 2nd byte: 0111xxxx
                # will be reformatted as
                # 1st byte: 00001111
                # 2nd byte: 11110111
                #loop from last to first byte in the buffer
                for i in range(len(byteBuffer)-1, -1, -1):
                    #from the second last byte in the buffer up to the first
                    if(i < len(byteBuffer)-1):

                        #move rigthmost shift bits all the way to the left side
                        extr = byteBuffer[i] & zeroizeMask
                        extr = extr << (8 - shift)
                        
                        #move that to the previous byte
                        byteBuffer[i+1] |= extr

                    #last byte in the buffer: shift right by rShift
                    byteBuffer[i] >>= shift

                #combine bytearray in one number
                valueToDecipher = int.from_bytes(byteBuffer, "big")
                
                #decypher value
                decypheredValue = RiceDecode(valueToDecipher, k)
                
                
                #write value to output buffer
                outBuffer.append(decypheredValue)
                
                #clean temporary bytearray
                byteBuffer = bytearray()
                
                #Exit condition - If true exit
                if(remByte == 0 and len(nextByte) == 0):
                     break
                        
                #add leftover byte to buffer for next iteration
                byteBuffer.append(remByte)
                startIdx = 8-shift
                    
                #If temporary buffer is not empty, reset temp buffer and current byte for next iteration
                #else read next byte from input
                if(len(byteBuffer) != 0):
                    sByte = byteBuffer
                    #reinitialize buffer
                    byteBuffer = bytearray()
                else:
                    sByte = nextByte
                    nextByte = sStream.read(1)

                #If output buffer is full, write to destination stream
                if(len(outBuffer) >= bufSize):
                    #write the buffer into the destination stream
                    dStream.write(outBuffer)
                    #empty the buffer
                    outBuffer = bytearray()


            #empty the buffer into the destination stream
            dStream.write(outBuffer)


            dStream.close()


In [9]:
originalFile = "Exercise2_Files\Sound2.wav"
encodedFile = "Exercise2_Files\Sound2_Enc.ex2"
decodedFile = "Exercise2_Files\Sound2_Enc_Dec.wav"


import filecmp

k = 7


print(f'Testing with K = {k}')

print('Encoding...')
EncodeFileWithRiceEncode(originalFile, encodedFile, k)
print("Done")

print('Decoding...')
DecodeFileWithRiceEncode(encodedFile, decodedFile, k)
print("Done")

comp = filecmp.cmp(originalFile, decodedFile)

print(f'Comparing original file and decoded file. Are they equal? {comp}\n\n')


Testing with K = 7
Encoding...
Done
Decoding...
Done
Comparing original file and decoded file. Are they equal? True


